In [1]:
import numpy as np
import pandas as pd

## 2017-2018 school vaccination data
- Numbers in the original document have been surpressed for <=1%, <=2%, <=5% and >=95%,>=98%,>=99%. Original file has errors with <=,  >=, and --* symbols when importing directly from .xlsx
- The 3 raw datafiles for childcare, kindergarden, and 7th grade has been compiled manually into a csv called `raw17_18_combined.csv` with <= and >= removed and replaced with the corresponding numbers

In [2]:
# Import raw combined 2017-2018 school vaccination data
pertusis_1718 = pd.read_csv("Raw_Data/raw17_18_combined.csv",encoding = "ISO-8859-1")

In [3]:
pertusis_1718.head()

,FACILITY_NUMBER,COUNTY,pub_priv_headstart,CITY,FACILITY_NAME,ENROLLMENT,n,pct,vac_info_type,REPORTED
0,10214883,ALAMEDA,HEAD START,ALAMEDA,ALAMEDA HEAD START . ANGELA AGUILAR CENTER,63.0,.,98,childcare,Y
1,13417425,ALAMEDA,HEAD START,ALAMEDA,ALAMEDA HEAD START . SUE MATHESON CENTER,47.0,.,95,childcare,Y
2,13420593,ALAMEDA,HEAD START,ALAMEDA,ALAMEDA HEAD START. COLLEGE OF ALAMEDA HEAD START,44.0,.,95,childcare,Y
3,13417441,ALAMEDA,PRIVATE,ALAMEDA,SUGAR AND SPICE,NaN,.,.,childcare,N
4,6151211,ALAMEDA,PRIVATE,ALAMEDA,MCKINNEY CHRISTIAN ACADEMY,NaN,.,.,kindergarten,N


In [4]:
#Clean up missing values and set to Nan
pertusis_1718.n=pertusis_1718.n.replace('.', np.nan)
pertusis_1718.pct=pertusis_1718.pct.replace('.',np.nan)

# uppercase all counties and cities
pertusis_1718.COUNTY=pertusis_1718['COUNTY'].str.upper()
pertusis_1718.CITY=pertusis_1718['CITY'].str.upper()

In [5]:
pertusis_1718.CITY

0            ALAMEDA
1            ALAMEDA
2            ALAMEDA
3            ALAMEDA
4            ALAMEDA
5            ALAMEDA
6            ALAMEDA
7            ALAMEDA
8            ALAMEDA
9            ALAMEDA
10           ALAMEDA
11           ALAMEDA
12           ALAMEDA
13           ALAMEDA
14           ALAMEDA
15           ALAMEDA
16           ALAMEDA
17           ALAMEDA
18           ALAMEDA
19           ALAMEDA
20           ALAMEDA
21           ALAMEDA
22           ALAMEDA
23           ALAMEDA
24           ALAMEDA
25           ALAMEDA
26           ALAMEDA
27           ALAMEDA
28           ALAMEDA
29           ALAMEDA
            ...     
19279     MARYSVILLE
19280     MARYSVILLE
19281     MARYSVILLE
19282     MARYSVILLE
19283     MARYSVILLE
19284     MARYSVILLE
19285     MARYSVILLE
19286     MARYSVILLE
19287     MARYSVILLE
19288     MARYSVILLE
19289     MARYSVILLE
19290     OLIVEHURST
19291     OLIVEHURST
19292     OLIVEHURST
19293     OLIVEHURST
19294     OLIVEHURST
19295     OLI

In [6]:
pertusis_1718.dtypes

FACILITY_NUMBER         int64
COUNTY                 object
pub_priv_headstart     object
CITY                   object
FACILITY_NAME          object
ENROLLMENT            float64
n                      object
pct                    object
vac_info_type          object
REPORTED               object
dtype: object

In [7]:
# Change n and percent to floats
pertusis_1718[['n', 'pct']] = pertusis_1718[['n', 'pct']].astype('float64')

In [8]:
# Check conversion
pertusis_1718.dtypes

FACILITY_NUMBER         int64
COUNTY                 object
pub_priv_headstart     object
CITY                   object
FACILITY_NAME          object
ENROLLMENT            float64
n                     float64
pct                   float64
vac_info_type          object
REPORTED               object
dtype: object

In [9]:
pertusis_1718.head()

,FACILITY_NUMBER,COUNTY,pub_priv_headstart,CITY,FACILITY_NAME,ENROLLMENT,n,pct,vac_info_type,REPORTED
0,10214883,ALAMEDA,HEAD START,ALAMEDA,ALAMEDA HEAD START . ANGELA AGUILAR CENTER,63.0,NaN,98.0,childcare,Y
1,13417425,ALAMEDA,HEAD START,ALAMEDA,ALAMEDA HEAD START . SUE MATHESON CENTER,47.0,NaN,95.0,childcare,Y
2,13420593,ALAMEDA,HEAD START,ALAMEDA,ALAMEDA HEAD START. COLLEGE OF ALAMEDA HEAD START,44.0,NaN,95.0,childcare,Y
3,13417441,ALAMEDA,PRIVATE,ALAMEDA,SUGAR AND SPICE,NaN,NaN,NaN,childcare,N
4,6151211,ALAMEDA,PRIVATE,ALAMEDA,MCKINNEY CHRISTIAN ACADEMY,NaN,NaN,NaN,kindergarten,N


In [10]:
sum(pertusis_1718.REPORTED=='N')

1005

In [11]:
sum(pertusis_1718.REPORTED=='Y')

18304

Subset dataset to those that reported

In [12]:
# Take subset that have numbers
vac_1718=pertusis_1718[pertusis_1718.REPORTED=='Y']

In [13]:
vac_1718.head()

,FACILITY_NUMBER,COUNTY,pub_priv_headstart,CITY,FACILITY_NAME,ENROLLMENT,n,pct,vac_info_type,REPORTED
0,10214883,ALAMEDA,HEAD START,ALAMEDA,ALAMEDA HEAD START . ANGELA AGUILAR CENTER,63.0,NaN,98.0,childcare,Y
1,13417425,ALAMEDA,HEAD START,ALAMEDA,ALAMEDA HEAD START . SUE MATHESON CENTER,47.0,NaN,95.0,childcare,Y
2,13420593,ALAMEDA,HEAD START,ALAMEDA,ALAMEDA HEAD START. COLLEGE OF ALAMEDA HEAD START,44.0,NaN,95.0,childcare,Y
6,13415928,ALAMEDA,PRIVATE,ALAMEDA,BRIGHT HORIZONS AT GARNER,109.0,NaN,99.0,childcare,Y
7,13419403,ALAMEDA,PRIVATE,ALAMEDA,PETER PAN SCHOOL,100.0,NaN,99.0,childcare,Y


Numbers in the original document have been surpressed for <=1%, <=2%, <=5% and >=95%,>=98%,>=99% so will impute values by assuming that the percentage is equal to whatever value listed and multiply by the number of students enrolled at the school. 

In [14]:
# Calculate n from enrollment and approximate percentage
vac_1718.loc[:,('n')]=vac_1718.n.fillna(vac_1718.ENROLLMENT*vac_1718.pct/100)

C:\Users\42\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [15]:
vac_1718.head()

,FACILITY_NUMBER,COUNTY,pub_priv_headstart,CITY,FACILITY_NAME,ENROLLMENT,n,pct,vac_info_type,REPORTED
0,10214883,ALAMEDA,HEAD START,ALAMEDA,ALAMEDA HEAD START . ANGELA AGUILAR CENTER,63.0,61.74,98.0,childcare,Y
1,13417425,ALAMEDA,HEAD START,ALAMEDA,ALAMEDA HEAD START . SUE MATHESON CENTER,47.0,44.65,95.0,childcare,Y
2,13420593,ALAMEDA,HEAD START,ALAMEDA,ALAMEDA HEAD START. COLLEGE OF ALAMEDA HEAD START,44.0,41.80,95.0,childcare,Y
6,13415928,ALAMEDA,PRIVATE,ALAMEDA,BRIGHT HORIZONS AT GARNER,109.0,107.91,99.0,childcare,Y
7,13419403,ALAMEDA,PRIVATE,ALAMEDA,PETER PAN SCHOOL,100.0,99.00,99.0,childcare,Y


In [16]:
# Change to integers 
vac_1718.loc[:,('ENROLLMENT','n','pct')]=vac_1718.loc[:,('ENROLLMENT','n','pct')].astype('int64')

C:\Users\42\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [17]:
vac_1718.head()

,FACILITY_NUMBER,COUNTY,pub_priv_headstart,CITY,FACILITY_NAME,ENROLLMENT,n,pct,vac_info_type,REPORTED
0,10214883,ALAMEDA,HEAD START,ALAMEDA,ALAMEDA HEAD START . ANGELA AGUILAR CENTER,63,61,98,childcare,Y
1,13417425,ALAMEDA,HEAD START,ALAMEDA,ALAMEDA HEAD START . SUE MATHESON CENTER,47,44,95,childcare,Y
2,13420593,ALAMEDA,HEAD START,ALAMEDA,ALAMEDA HEAD START. COLLEGE OF ALAMEDA HEAD START,44,41,95,childcare,Y
6,13415928,ALAMEDA,PRIVATE,ALAMEDA,BRIGHT HORIZONS AT GARNER,109,107,99,childcare,Y
7,13419403,ALAMEDA,PRIVATE,ALAMEDA,PETER PAN SCHOOL,100,99,99,childcare,Y


In [18]:
# Change print settings to see all
#np.set_printoptions(threshold=np.nan)
#change back to default printing length
np.set_printoptions(threshold=5)
# Check County names
sorted(list(vac_1718.COUNTY.unique()))

['ALAMEDA',
 'AMADOR',
 'BUTTE',
 'CALAVERAS',
 'COLUSA',
 'CONTRA COSTA',
 'DEL NORTE',
 'EL DORADO',
 'FRESNO',
 'GLENN',
 'HUMBOLDT',
 'IMPERIAL',
 'INYO',
 'KERN',
 'KINGS',
 'LAKE',
 'LASSEN',
 'LOS ANGELES',
 'MADERA',
 'MARIN',
 'MARIPOSA',
 'MENDOCINO',
 'MERCED',
 'MODOC',
 'MONO',
 'MONTEREY',
 'NAPA',
 'NEVADA',
 'ORANGE',
 'PLACER',
 'PLUMAS',
 'RIVERSIDE',
 'SACRAMENTO',
 'SAN BENITO',
 'SAN BERNARDINO',
 'SAN DIEGO',
 'SAN FRANCISCO',
 'SAN JOAQUIN',
 'SAN LUIS OBISPO',
 'SAN MATEO',
 'SANTA BARBARA',
 'SANTA CLARA',
 'SANTA CRUZ',
 'SHASTA',
 'SIERRA',
 'SISKIYOU',
 'SOLANO',
 'SONOMA',
 'STANISLAUS',
 'SUTTER',
 'TEHAMA',
 'TRINITY',
 'TULARE',
 'TUOLUMNE',
 'VENTURA',
 'YOLO',
 'YUBA']

In [19]:
#Check city names
sorted(list(vac_1718.CITY.unique()))

['ACAMPO',
 'ACTON',
 'ACTON, CA',
 'ADELANTO',
 'AGOURA',
 'AGOURA HILLS',
 'AGUANGA',
 'AHWAHNEE',
 'ALAMEDA',
 'ALAMO',
 'ALBANY',
 'ALHAMBRA',
 'ALISO VIEJO',
 'ALPAUGH',
 'ALPINE',
 'ALTA LOMA',
 'ALTADENA',
 'ALTURAS',
 'ALVISO',
 'AMERICAN CANYON',
 'ANAHEIM',
 'ANAHEIM HILLS',
 'ANAHEIM,',
 'ANANHEIM',
 'ANDERSON',
 'ANGELS CAMP',
 'ANTELOPE',
 'ANTIOCH',
 'ANZA',
 'APPLE VALLEY',
 'APTOS',
 'ARBOGA',
 'ARBUCKLE',
 'ARCADIA',
 'ARCATA',
 'ARLETA',
 'ARMONA',
 'ARNOLD',
 'AROMAS',
 'ARROYO GRANDE',
 'ARTESIA',
 'ARVIN',
 'ATASCADERO',
 'ATHERTON',
 'ATWATER',
 'AUBERRY',
 'AUBURN',
 'AVALON',
 'AVENAL',
 'AVERY',
 'AZUSA',
 'BAKERSFIELD',
 'BALDWIN PARK',
 'BALLICO',
 'BANNING',
 'BANTA',
 'BARSTOW',
 'BAY POINT',
 'BAYSIDE',
 'BEALE AFB',
 'BEALE AIR FORCE BASE',
 'BEAUMONT',
 'BELL',
 'BELL GARDENS',
 'BELLA VISTA',
 'BELLFLOWER',
 'BELMONT',
 'BELVEDERE',
 'BENICA',
 'BENICIA',
 'BERKELEY',
 'BERMUDA DUNES',
 'BEVERLY HILLS',
 'BIG BEAR CITY',
 'BIG BEAR LAKE',
 'BIGGS',
 'BI

In [20]:
# Fix City Typos
vac_1718.loc[vac_1718.CITY=='ACTON, CA', 'CITY']='ACTON'
vac_1718.loc[vac_1718.CITY=='AGOURA', 'CITY']='AGOURA HILLS'
vac_1718.loc[vac_1718.CITY=='ANAHEIM,', 'CITY']='ANAHEIM'
vac_1718.loc[vac_1718.CITY=='ANANHEIM,', 'CITY']='ANAHEIM'
vac_1718.loc[vac_1718.CITY=='CA', 'CITY']='NAPA'
vac_1718.loc[vac_1718.CITY=='BEALE AIR FORCE BASE', 'CITY']='BEALE AFB' # to match census name
vac_1718.loc[vac_1718.CITY=='BENICA', 'CITY']='BENICIA'
vac_1718.loc[vac_1718.CITY=='CARDIFF', 'CITY']='CARDIFF BY THE SEA'
vac_1718.loc[vac_1718.CITY=='CARMEL', 'CITY']='CARMEL VALLEY'
vac_1718.loc[vac_1718.CITY=='CHINO,', 'CITY']='CHINO'
vac_1718.loc[vac_1718.CITY=='CHULAR', 'CITY']='CHUALAR'
vac_1718.loc[vac_1718.CITY=='CITY OF COMMERCE', 'CITY']='COMMERCE'
vac_1718.loc[vac_1718.CITY=='CITY OF INDUSTRY', 'CITY']='INDUSTRY'
vac_1718.loc[vac_1718.CITY=='CUDAHAY', 'CITY']='CUDAHY'
vac_1718.loc[vac_1718.CITY=='E. NICOLAUS', 'CITY']='EAST NICOLAUS'
vac_1718.loc[vac_1718.CITY=='E. RANCHO DOMINGUEZ', 'CITY']='EAST RANCHO DOMINGUEZ'
vac_1718.loc[vac_1718.CITY=='E. WHITTIER', 'CITY']='EAST WHITTIER'
vac_1718.loc[vac_1718.CITY=='EL SEGUNDO,', 'CITY']='EL SEGUNDO'
vac_1718.loc[vac_1718.CITY=='FAIRIFELD', 'CITY']='FAIRFIELD'
vac_1718.loc[vac_1718.CITY=='FREMOTN', 'CITY']='FREMONT'
vac_1718.loc[vac_1718.CITY=='FT. IRWIN', 'CITY']='FORT IRWIN'
vac_1718.loc[vac_1718.CITY=='GREENFILED', 'CITY']='GREENFIELD'
vac_1718.loc[vac_1718.CITY=='HUNTINGTON', 'CITY']='HUNTINGTON PARK'
vac_1718.loc[vac_1718.CITY=='JAMUAL', 'CITY']='JAMUL'
vac_1718.loc[vac_1718.CITY=='LA', 'CITY']='LOS ANGELES'
vac_1718.loc[vac_1718.CITY=='LA CANADA', 'CITY']='LA CANADA FLINTRIDGE'
vac_1718.loc[vac_1718.CITY=='LAKE VIEW TERRANCE', 'CITY']='LAKE VIEW TERRACE'
vac_1718.loc[vac_1718.CITY=='LAKEVIEW TERRACE', 'CITY']='LAKE VIEW TERRACE'
vac_1718.loc[vac_1718.CITY=='LANCASTER,', 'CITY']='LANCASTER'
vac_1718.loc[vac_1718.CITY=='LAVERNE', 'CITY']='LA VERNE'
vac_1718.loc[vac_1718.CITY=='MC KINLEYVILLE', 'CITY']='MCKINLEYVILLE'
vac_1718.loc[vac_1718.CITY=='MONTROSE', 'CITY']='LA CRESCENTA-MONTROSE'
vac_1718.loc[vac_1718.CITY=='MT. SHASTA', 'CITY']='MOUNT SHASTA'
vac_1718.loc[vac_1718.CITY=='N. HOLLYWOOD', 'CITY']='NORTH HOLLYWOOD'
vac_1718.loc[vac_1718.CITY=='N.A.S. LEMOORE', 'CITY']='LEMOORE STATION' # match census
vac_1718.loc[vac_1718.CITY=='NAS LEMOORE', 'CITY']='LEMOORE STATION'
vac_1718.loc[vac_1718.CITY=='NEWPORT BEACH,', 'CITY']='NEWPORT BEACH'
vac_1718.loc[vac_1718.CITY=='PACOMIA', 'CITY']='PACOIMA'
vac_1718.loc[vac_1718.CITY=='PANORAM ', 'CITY']='PANORAMA CITY'
vac_1718.loc[vac_1718.CITY=='PT. REYES STATION', 'CITY']='POINT REYES STATION'
vac_1718.loc[vac_1718.CITY=='RANCHO SANTA MARGARI', 'CITY']='RANCHO SANTA MARGARITA'
vac_1718.loc[vac_1718.CITY=='RANCHO SAN MARGARITA', 'CITY']='RANCHO SANTA MARGARITA'
vac_1718.loc[vac_1718.CITY=='RANCHO STA MARGAITA', 'CITY']='RANCHO SANTA MARGARITA'
vac_1718.loc[vac_1718.CITY=='RANCHO STA MARGARITA', 'CITY']='RANCHO SANTA MARGARITA'
vac_1718.loc[vac_1718.CITY=='RCHO STA MARG', 'CITY']='RANCHO SANTA MARGARITA'
vac_1718.loc[vac_1718.CITY=='RANCHOS PALOS VERDES', 'CITY']='RANCHO PALOS VERDES'
vac_1718.loc[vac_1718.CITY=='RIVERSIDE,', 'CITY']='RIVERSIDE'
vac_1718.loc[vac_1718.CITY=='ROLLING HILLS ESTATE', 'CITY']='ROLLING HILLS ESTATES'
vac_1718.loc[vac_1718.CITY=='S. EL MONTE', 'CITY']='SOUTH EL MONTE'
vac_1718.loc[vac_1718.CITY=='S. LAKE TAHOE', 'CITY']='SOUTH LAKE TAHOE'
vac_1718.loc[vac_1718.CITY=='SAN BERARDINO', 'CITY']='SAN BERNARDINO'
vac_1718.loc[vac_1718.CITY=='SAN FRNCISCO', 'CITY']='SAN FRANCISCO'
vac_1718.loc[vac_1718.CITY=='SAN JOSE,', 'CITY']='SAN JOSE'
vac_1718.loc[vac_1718.CITY=='SANTA BARARA', 'CITY']='SANTA BARBARA'
vac_1718.loc[vac_1718.CITY=='SHASTA LAKE CITY', 'CITY']='SHASTA LAKE'
vac_1718.loc[vac_1718.CITY=='SILVERADO CANYON', 'CITY']='SILVERADO'
vac_1718.loc[vac_1718.CITY=='SOUTH LATE TAHOE', 'CITY']='SOUTH LAKE TAHOE'
vac_1718.loc[vac_1718.CITY=='SPRECKLES', 'CITY']='SPRECKELS'
vac_1718.loc[vac_1718.CITY=='STEVENSONS RANCH', 'CITY']='STEVENSON RANCH'
vac_1718.loc[vac_1718.CITY=='SUISUN', 'CITY']='SUISUN CITY'
vac_1718.loc[vac_1718.CITY=='SUNNYALE', 'CITY']='SUNNYVALE'
vac_1718.loc[vac_1718.CITY=='SUPELVEDA', 'CITY']='SEPULVEDA'
vac_1718.loc[vac_1718.CITY=='TURLOCK,', 'CITY']='TURLOCK'
vac_1718.loc[vac_1718.CITY=='UPPERLAKE', 'CITY']='UPPER LAKE'
vac_1718.loc[vac_1718.CITY=='VANDENBERG AIR FORCE BASE', 'CITY']='VANDENBERG AFB' # match census
vac_1718.loc[vac_1718.CITY=='W. SACRAMENTO', 'CITY']='WEST SACRAMENTO'

C:\Users\42\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [21]:
# Change case to title case
vac_1718.COUNTY=vac_1718['COUNTY'].str.title()
vac_1718.CITY=vac_1718['CITY'].str.title()

C:\Users\42\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [22]:
#Get count of students vaccinated by county,city and grade
vac_n=vac_1718.groupby(by=['COUNTY','CITY','vac_info_type'], as_index=False)['n'].sum()
vac_n

,COUNTY,CITY,vac_info_type,n
0,Alameda,Alameda,7th grade,931
1,Alameda,Alameda,childcare,1231
2,Alameda,Alameda,kindergarten,986
3,Alameda,Albany,7th grade,255
4,Alameda,Albany,childcare,249
5,Alameda,Albany,kindergarten,307
6,Alameda,Berkeley,7th grade,908
7,Alameda,Berkeley,childcare,2346
8,Alameda,Berkeley,kindergarten,860
9,Alameda,Castro Valley,7th grade,776


In [25]:
#Export count of students vaccinated by county,city and grade
vac_n.to_csv("vac_n_table1718.CSV")

In [28]:
#Get average percentage of students vaccinated by county,city and grade
vac_pct=vac_1718.groupby(by=['COUNTY','CITY','vac_info_type'], as_index=False)['pct'].mean()
vac_pct

,COUNTY,CITY,vac_info_type,pct
0,Alameda,Alameda,7th grade,94.900000
1,Alameda,Alameda,childcare,96.000000
2,Alameda,Alameda,kindergarten,96.384615
3,Alameda,Albany,7th grade,98.000000
4,Alameda,Albany,childcare,95.571429
5,Alameda,Albany,kindergarten,97.666667
6,Alameda,Berkeley,7th grade,96.750000
7,Alameda,Berkeley,childcare,92.750000
8,Alameda,Berkeley,kindergarten,91.333333
9,Alameda,Castro Valley,7th grade,97.000000


In [29]:
#Export average percentage of students vaccinated by county,city and grade
vac_pct.to_csv("vac_pct_table1718.CSV")

In [30]:
vac_1718.head()

,FACILITY_NUMBER,COUNTY,pub_priv_headstart,CITY,FACILITY_NAME,ENROLLMENT,n,pct,vac_info_type,REPORTED
0,10214883,Alameda,HEAD START,Alameda,ALAMEDA HEAD START . ANGELA AGUILAR CENTER,63,61,98,childcare,Y
1,13417425,Alameda,HEAD START,Alameda,ALAMEDA HEAD START . SUE MATHESON CENTER,47,44,95,childcare,Y
2,13420593,Alameda,HEAD START,Alameda,ALAMEDA HEAD START. COLLEGE OF ALAMEDA HEAD START,44,41,95,childcare,Y
6,13415928,Alameda,PRIVATE,Alameda,BRIGHT HORIZONS AT GARNER,109,107,99,childcare,Y
7,13419403,Alameda,PRIVATE,Alameda,PETER PAN SCHOOL,100,99,99,childcare,Y


In [31]:
#Export full cleaned data
vac_1718.to_csv("clean1718.csv", index=False)

## Census Data Cleaning

### Demographics

In [ ]:
# Import and clean census data for basic demographics age gender race
census16 = pd.read_csv("Raw_Data/Census/ACS_16_demographic.csv",encoding = "ISO-8859-1",header=[0],na_values=['-'],)

In [ ]:
census16.head()

In [ ]:
demographic16 = census16[['GEO.id','GEO.id2','GEO.display-label',
                     'HC01_VC03',
                     'HC01_VC04','HC03_VC04',
                     'HC01_VC05','HC03_VC05',
                     'HC01_VC08','HC03_VC08',
                     'HC01_VC09','HC03_VC09',
                     'HC01_VC10','HC03_VC10',
                     'HC01_VC11','HC03_VC11',
                     'HC01_VC12','HC03_VC12',
                     'HC01_VC13','HC03_VC13',
                     'HC01_VC14','HC03_VC14',
                     'HC01_VC15','HC03_VC15',
                     'HC01_VC16','HC03_VC16',
                     'HC01_VC17','HC03_VC17',
                     'HC01_VC18','HC03_VC18',
                     'HC01_VC19','HC03_VC19',
                     'HC01_VC20','HC03_VC20',
                     'HC01_VC23','HC03_VC23',
                     'HC01_VC88','HC03_VC88',
                     'HC01_VC94','HC03_VC94',
                     'HC01_VC95','HC03_VC95',
                     'HC01_VC96','HC03_VC96',
                     'HC01_VC97','HC03_VC97',
                     'HC01_VC98','HC03_VC98',
                     'HC01_VC99','HC03_VC99',
                     'HC01_VC100','HC03_VC100',
                     'HC01_VC101','HC03_VC101',
                     'HC01_VC102','HC03_VC102'                 
        ]]

In [ ]:
demographic16.dtypes

In [ ]:
# demographic16[['HC01_VC03','HC01_VC04','HC01_VC05','HC01_VC08','HC01_VC09','HC01_VC10','HC01_VC11','HC01_VC12','HC01_VC13',
#           'HC01_VC14','HC01_VC15','HC01_VC16','HC01_VC17','HC01_VC18','HC01_VC19','HC01_VC20','HC01_VC23','HC01_VC88',
#           'HC01_VC94','HC01_VC95','HC01_VC96','HC01_VC97','HC01_VC98',
#           'HC01_VC99','HC01_VC100','HC01_VC101','HC01_VC102']] = demographic16[['HC01_VC03','HC01_VC04','HC01_VC05','HC01_VC08','HC01_VC09','HC01_VC10','HC01_VC11','HC01_VC12','HC01_VC13',
#           'HC01_VC14','HC01_VC15','HC01_VC16','HC01_VC17','HC01_VC18','HC01_VC19','HC01_VC20','HC01_VC23','HC01_VC88',
#           'HC01_VC94','HC01_VC95','HC01_VC96','HC01_VC97','HC01_VC98',
#           'HC01_VC99','HC01_VC100','HC01_VC101','HC01_VC102']].astype('float64')

In [ ]:
#combine other race, two or more races (HC01_VC99, HC01_VC100, HC01_VC101, HC01_VC102)
demographic16['other']=pd.Series(census16['HC01_VC99']+census16['HC01_VC100']+census16['HC01_VC101']+census16['HC01_VC102'])

In [ ]:
demographic16['other_pct']=pd.Series(census16['HC03_VC99']+census16['HC03_VC100']+census16['HC03_VC101']+census16['HC03_VC102'])

In [ ]:
demographic16.rename(columns={'GEO.display-label': 'city',
                         'HC01_VC03': 'tot_pop', 
                         'HC01_VC04': 'male',
                         'HC01_VC05': 'female',
                         'HC01_VC08':'under_5',
                         'HC01_VC09':'5_9',
                         'HC01_VC10':'10_14',
                         'HC01_VC11':'15_19',
                         'HC01_VC12':'20_24',
                         'HC01_VC13':'25_34',
                         'HC01_VC14':'35_44',
                         'HC01_VC15':'45_54',
                         'HC01_VC16':'55_59',
                         'HC01_VC17':'60_64',
                         'HC01_VC18':'65_74',
                         'HC01_VC19':'75_84',
                         'HC01_VC20':'85_over',
                         'HC01_VC23':'median_age',
                         'HC01_VC88':'hispanic_latino',
                         'HC01_VC89':'mexican',
                         'HC01_VC94':'white',
                         'HC01_VC95':'black',
                         'HC01_VC96':'aian',
                         'HC01_VC97':'asian',
                         'HC01_VC98':'nhopi',

                         'HC03_VC04': 'male_pct',
                         'HC03_VC05': 'female_pct',
                         'HC03_VC08':'under_5_pct',
                         'HC03_VC09':'5_9_pct',
                         'HC03_VC10':'10_14_pct',
                         'HC03_VC11':'15_19_pct',
                         'HC03_VC12':'20_24_pct',
                         'HC03_VC13':'25_34_pct',
                         'HC03_VC14':'35_44_pct',
                         'HC03_VC15':'45_54_pct',
                         'HC03_VC16':'55_59_pct',
                         'HC03_VC17':'60_64_pct',
                         'HC03_VC18':'65_74_pct',
                         'HC03_VC19':'75_84_pct',
                         'HC03_VC20':'85_over_pct',
                         'HC03_VC23':'median_age_pct',
                         'HC03_VC88':'hispanic_latino_pct',
                         'HC03_VC89':'mexican_pct',
                         'HC03_VC94':'white_pct',
                         'HC03_VC95':'black_pct',
                         'HC03_VC96':'aian_pct',
                         'HC03_VC97':'asian_pct',
                         'HC03_VC98':'nhopi_pct'
                        }, 
                inplace=True)
# Other race categories
#                          'HC01_VC99':'other_only',
#                          'HC01_VC100':'more_two_races',
#                          'HC01_VC101':'more_two_races_other',
#                          'HC01_VC102':'more_two_races_other_exclude',


In [ ]:
demographic16.dtypes

In [ ]:
demographic16['city']=demographic16.city.replace(' city, California', '', regex=True)

In [ ]:
demographic16['city']=demographic16.city.replace(' CDP, California', '', regex=True)

In [ ]:
demographic16['city']=demographic16.city.replace(' town, California', '', regex=True)

In [ ]:
## Uppercase cities
demographic16['city']=demographic16['city'].str.upper()

In [ ]:
demographic16=demographic16.drop(columns=['HC01_VC99','HC03_VC99','HC01_VC100','HC03_VC100','HC01_VC101','HC03_VC101','HC01_VC102','HC03_VC102'])
demographic16

### Median Household Income and Health insurance Status

Median Household Income has categories 2,500-, 250,000+ which was changed to 2500 and 250000. 

In [ ]:
# Import and clean acs data for income and insurance status
acs16 = pd.read_csv("Raw_Data/Census/ACS_16_economic.csv", encoding = "ISO-8859-1", header=[0], na_values=['-']) 

In [ ]:
acs16.head()

In [ ]:
acs16=acs16[['GEO.id','GEO.id2','GEO.display-label','HC01_VC85','HC01_VC131','HC03_VC131',
                'HC01_VC132','HC03_VC132','HC01_VC133','HC03_VC133','HC01_VC134','HC03_VC134']]
income16=acs16

In [ ]:
income16.head()

In [ ]:
income16.dtypes

In [ ]:
income16.rename(columns={'GEO.display-label': 'city',
                         'HC01_VC85': 'median_income',
                         'HC01_VC131': 'insurance',
                         'HC01_VC132': 'private_insure',
                         'HC01_VC133': 'public_insure',
                         'HC01_VC134': 'no_insurance',

                         'HC03_VC131': 'insurance_pct',
                         'HC03_VC132': 'private_insure_pct',
                         'HC03_VC133': 'public_insure_pct',
                         'HC03_VC134': 'no_insurance_pct'  
                        }, 
                inplace=True)

In [ ]:
income16.dtypes

In [ ]:
income16['city']=income16.city.replace(' CDP, California', '', regex=True)

In [ ]:
income16['city']=income16.city.replace(' city, California', '', regex=True)

In [ ]:
income16['city']=income16.city.replace(' town, California', '', regex=True)

In [ ]:
income16.head()

## Join Demographics and income data

In [ ]:
pop_data=demographic16.merge(income16, left_on='GEO.id', right_on='GEO.id', how='outer')
pop_data

In [ ]:
list(pop_data.columns.values)

In [ ]:
pop_data=pop_data.drop(columns=['GEO.id2_y','city_y'])

In [ ]:
pop_data.rename(columns={'GEO.id': 'geoid',
                         'GEO.id2_x': 'geoid2',
                         'city_x': 'city'})

In [ ]:
# Exported merged census data
pop_data.to_csv("pop_data16.csv", index=False)

## Join Vaccine data to Census data

Join needs unique cities in the vaccine data to use as a key. Will need to figure out how to weight the different grade levels based on the census population numbers for age. 

In [ ]:
vac_pop_1718=vac_1718.merge(pop_data, left_on='CITY', right_on='city', how='outer')
vac_pop_1718